# Daily Load Profile Timeseries Clustering Evaluation

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from math import ceil

import plotly.plotly as py
import plotly.offline as po
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.tools as tools
import colorlover as cl
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()

import matplotlib. pyplot as plt
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap

import evaluation.eval_clusters as ec
import evaluation.eval_cluster_plot as pc
from support import data_dir
eval_dir = os.path.join(data_dir,'cluster_evaluation')

In [ ]:
experiments = ec.getExperiments('exp')
exp2 = ec.getExperiments('exp2')
exp3 = ec.getExperiments('exp3')
exp4 = ec.getExperiments('exp4')
exp5 = ec.getExperiments('exp5')
exp6 = ec.getExperiments('exp6')
#experiments

## Analyse Cluster Scores

### Davies-Bouldin Index

In [ ]:
pc.plotClusterIndex('dbi', 'Davies-Bouldin Index', experiments, groupby='algorithm')

### Mean Index Adequacy

In [ ]:
pc.plotClusterIndex('mia','Mean Index Adequacy', experiments)

### Silhouette Score

The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.

In [ ]:
pc.plotClusterIndex('silhouette', 'Silhouette Score', experiments, groupby='algorithm')

### Combined Cluster Score

In [ ]:
pc.plotClusterIndex('score','Combined Cluster Score', experiments, groupby='algorithm', ylog=True)

## Explore Cluster Labels, Centroids and Sizes

### Select best clusters for different algorithms

In [ ]:
cluster_results = ec.readResults()

selected_clusters = dict()
for e in experiments:
    clusters = ec.selectClusters(cluster_results, 5, threshold=1200, experiment=e)
    selected_clusters[e] = clusters
selected_clusters['best'] = ec.selectClusters(cluster_results, 10, threshold=1200)

In [ ]:
selected_clusters['best']

### Get Cluster Labels

In [ ]:
best_exp = ['exp2_kmeans_unit_norm', 'exp4_kmeans_unit_norm', 'exp5_kmeans_unit_norm', 'exp6_kmeans_unit_norm']
            #'exp6_kmeans_demin', 'exp2_kmeans_demin',]
            #'exp4_kmeans_zero-one', 'exp5_kmeans_zero-one']

In [ ]:
cluster_labels = dict()
for e in best_exp[-1:]:
    labels = ec.getLabels(e)
    cluster_labels[e] = labels

### Get denormalised (real) cluster centroids

In [ ]:
real_cluster_centroids = dict()

for e in best_exp:
    rccpath = os.path.join(eval_dir, 'best_centroids', e +'BEST1_centroids.csv')
    centroids  = pd.read_csv(rccpath, index_col='k')
    real_cluster_centroids[e] = centroids

In [ ]:
real_cluster_centroids['exp2_kmeans_unit_norm']

### Get normalised cluster centroids 
#### only for exp2 & exp3

In [ ]:
norm_cluster_centroids = dict()
for k, v in selected_clusters.items():
    name = k
    centroids, cs, meta  = ec.getCentroids(v)
    norm_cluster_centroids[name] = {'centroids':centroids, 'cluster_size':cs, 'description':meta}

### Visualise Centroids

In [ ]:
ec.exploreAMDBins(cluster_results, 'exp5_kmeans_unit_norm')

In [ ]:
ec.exploreAMDBins(cluster_results, 'exp6_kmeans_unit_norm')

In [ ]:
pc.plotClusterCentroids(real_cluster_centroids[best_exp[0]])
pc.plotClusterCentroids(real_cluster_centroids[best_exp[1]])
pc.plotClusterCentroids(real_cluster_centroids[best_exp[2]])

## Explore Patterns in Cluster Labels

### Visualise Cluster Label Assignment

In [ ]:
pc.plotClusterLabels(list(cluster_labels.values())[2], 2014)

### Visualise TEMPORAL Cluster Specificity

In [ ]:
i = 0
pc.plotClusterSpecificity(best_exp[i], corr_list=['daytype','weekday'], threshold=1200, relative=[[5,1,1],1])
pc.plotClusterSpecificity(best_exp[i], corr_list=['season','monthly'], threshold=1200, relative=[[8, 4],1])
pc.plotClusterSpecificity(best_exp[i], corr_list=['yearly'], threshold=1200)

### Visualise CONTEXTUAL Cluster Specificity (Daily Demand Assignment)

In [ ]:
experiment = 'exp2_kmeans_unit_norm'

corr_path = os.path.join(data_dir, 'cluster_evaluation', 'k_correlations')

dif = pd.read_csv(os.path.join(corr_path, 'demandi_corr.csv'), index_col=[0,1,2], header=[0]).drop_duplicates()
dif_temp = dif.reset_index(level=[-2,-1])
int100_total = dif_temp[(dif_temp.experiment==experiment+'BEST1')&(dif_temp.compare=='total')].drop(['experiment','compare'],axis=1)

dqf = pd.read_csv(os.path.join(corr_path, 'demandq_corr.csv'), index_col=[0,1,2], header=[0]).drop_duplicates()
dqf_temp = dqf.reset_index(level=[-2,-1])
q100_total = dqf_temp[(dqf_temp.experiment==experiment+'BEST1')&(dqf_temp.compare=='total')].drop(['experiment','compare'],axis=1)

In [ ]:
#Equally spaced daily demand intervals
i = int100_total.T.stack().reset_index()
i.columns = ['int100_bins', 'cluster', 'values']
fig = i.iplot(kind='heatmap', x = 'int100_bins', y='cluster', z='values', colorscale='Reds', 
              title= 'Heatmap of relative likelihood of Cluster k being used in consumption bin', asFigure=True)
fig['layout']['xaxis'].update(dict(title = 'total daily demand bins (Amps)', 
                                   tickmode='array', tickvals=list(range(0,100,10)), ticktext = list(range(0,1000,100))))
fig['layout']['yaxis'].update(dict(title='Cluster k'))
po.iplot(fig)

#Equally sized daily demand intervals (quantiles)
rel_q100 = q100_total.T.drop(columns=33)/0.01

slatered=['#232c2e', '#ffffff','#c34513']
label_cmap, label_cs = pc.colorscale_from_list(slatered, 'label_cmap') 
colorscl= pc.asymmetric_colorscale(rel_q100, label_cmap, ref_point=1.0)

heatmap = go.Heatmap(z = rel_q100.T.values, x = rel_q100.index, y = rel_q100.columns, name = 'corr', 
                          colorscale=colorscl)
layout = go.Layout(
        title= 'Heatmap of relative likelihood of Cluster k being used in consumption quantile',
        xaxis=dict(title = 'total daily demand quantiles (Amps) - log scale', type='log'),
        yaxis=dict(title ='Cluster k'))
fig = {'data':[heatmap], 'layout':layout }
po.iplot(fig)

## Analyse Cluster Representativity and Specificity

In [ ]:
total_consE, peak_consE, peak_coincR, temporal_entropy, context_entropy = ec.getMeasures(best_exp, 1200)

### Consumption Error - total

In [ ]:
pc.plotClusterMetrics(total_consE, 'TOTAL consumption error evaluation metrics')

### Consumption Error - max

In [ ]:
pc.plotClusterMetrics(peak_consE, 'PEAK consumption error evaluation metrics')

### Peak Coincidence Ratio

In [ ]:
pc.plotClusterMetrics(peak_coincR, 'daily peak coincidence ratios', metric='coincidence_ratio', make_area_plot=True)

### Cluster Entropy - TEMPORAL
#### weekday, month

In [ ]:
pc.plotClusterMetrics(temporal_entropy, 'time-derived cluster entropy')#, metric='weekday_entropy', make_area_plot=False )

### Cluster Entropy - CONTEXTUAL
#### total daily demand, max daily demand

In [ ]:
pc.plotClusterMetrics(context_entropy, 'context-derived cluster entropy')

In [ ]:
#ec.saveMeasures(best_exp, 1200)
pd.read_csv(os.path.join(eval_dir,'cluster_entropy.csv'), index_col=[0,1], header=[0,1,2])